In [1]:
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf

IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3

def read_and_decode(filename, reshape_dims):
  # Read the file
  img = tf.io.read_file(filename)
  # Convert the compressed string to a 3D uint8 tensor.
  img = tf.image.decode_jpeg(img, channels=IMG_CHANNELS)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # Resize the image to the desired size.
  return tf.image.resize(img, reshape_dims)

CLASS_NAMES = [item.numpy().decode("utf-8") for item in 
               tf.strings.regex_replace(
                 tf.io.gfile.glob(r"E:\\computer vision\\practical-cv\\dataset\\train/*"),
                 r'E:\\computer vision\\practical-cv\\dataset\\train/', "")]
CLASS_NAMES = [item for item in CLASS_NAMES if item.find(".") == -1]
print("These are the available classes:", CLASS_NAMES)



These are the available classes: ['E:\\\\computer vision\\\\practical-cv\\\\dataset\\\\train\\daisy', 'E:\\\\computer vision\\\\practical-cv\\\\dataset\\\\train\\dandelion', 'E:\\\\computer vision\\\\practical-cv\\\\dataset\\\\train\\rose', 'E:\\\\computer vision\\\\practical-cv\\\\dataset\\\\train\\sunflower', 'E:\\\\computer vision\\\\practical-cv\\\\dataset\\\\train\\tulip']


In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
def decode_csv(csv_row):
  record_defaults = ["path", "flower"]
  filename, label_string = tf.io.decode_csv(csv_row, record_defaults)
  img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
  # label = tf.math.equal(CLASS_NAMES, label_string)
  return img, label_string

dataset = (tf.data.TextLineDataset(
    "E:\\computer vision\\practical-cv\\dataset\\train_csv.csv").map(decode_csv))

for img, label in dataset.take(1):
  avg = tf.math.reduce_mean(img, axis=[0, 1]) # average pixel in the image
  print(label, avg)

tf.Tensor(b'daisy', shape=(), dtype=string) tf.Tensor([0.6474569  0.6199989  0.50390774], shape=(3,), dtype=float32)


In [3]:
def decode_csv(csv_row):
  record_defaults = ["path", "flower"]
  filename, label_string = tf.io.decode_csv(csv_row, record_defaults)
  img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
  label = tf.argmax(tf.math.equal(CLASS_NAMES, label_string))
  return img, label

In [4]:
train_dataset = (tf.data.TextLineDataset(
    "E:\\computer vision\\practical-cv\\dataset\\train_csv.csv").
    map(decode_csv)).batch(3)

In [5]:
test_dataset = (tf.data.TextLineDataset(
    "E:\\computer vision\\practical-cv\\dataset\\val.csv").
    map(decode_csv)).batch(3)

In [22]:
"""
in this function i basiclly defining our nural network with learning rate and regulraization 
"""



def train_and_evaluate(batch_size = 3,
                       lrate = 0.001, # default in Adam constructor
                       l1 = 0,
                       l2 = 0,
                       num_hidden = 12):
    
    regulrizer = tf.keras.regularizers.l1_l2(l1, l2)

    train_dataset = (tf.data.TextLineDataset(
            "E:\\computer vision\\practical-cv\\dataset\\train_csv.csv").map(decode_csv)).batch(batch_size),

    test_dataset = (tf.data.TextLineDataset(
            "E:\\computer vision\\practical-cv\\dataset\\val.csv").map(decode_csv)).batch(batch_size),

    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape =  (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
        tf.keras.layers.Dense(num_hidden, kernel_regularizer= regulrizer, activation= tf.keras.activations.relu),
        tf.keras.layers.Dense(len(CLASS_NAMES), kernel_regularizer= regulrizer, activation='softmax')
    ])

    model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=lrate),
                  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])
    history = model.fit(train_dataset, validation_data=test_dataset, epochs=1)

    return model

In [25]:
model = train_and_evaluate(batch_size=3, lrate=0.0001, l1=0, l2=0, num_hidden=12)

### hyperparameter tuning

In [ ]:
"""
Hyperparameter tuning is the process of finding the configuration of hyperparameters that results in the best performance.
Hyperparameters are the variables that control the training process and the topology of an ML model."""
#we use the hyperparameter tuning when we fit the model

import tensorflow as tf
def build_model(hp):    #hp is the hyperparameter
    lrate = hp.Float('lrate', 1e-4, 1e-2, sampling = 'log')
    l1 = 0
    l2 = hp.Choice('l2', values=[0.0, 1e-1, 1e-2, 1e-3, 1e-4])
    num_hidden = hp.Int('num_hidden', min_value = 32, max_value = 512, step = 32)

    regulazier = tf.keras.regularizers.L1L2(l1 = l1, l2 = l2)
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
        tf.keras.layers.Dense(num_hidden, kernal_regularizer = regulazier, activation = 'relu'),
        tf.keras.layers.Dense(len(CLASS_NAMES), kernal_regularizer = regulazier, activation = 'softmax')
    ])

    #class name is represent the no. of classes are present in our 5-flower dataset

    model.compile(
        optimizer = tf.keras.optimizer.Adam(learning_rate = lrate),
        loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
        metrics = ['accuracy']
    )
    return model


train_dataset = (tf.data.TextLineDataset(
        "E:\\computer vision\\practical-cv\\dataset\\train_csv.csv").map(decode_csv)).batch(3)

test_dataset = (tf.data.TextLineDataset(
    "E:\\computer vision\\practical-cv\\dataset\\val.csv").map(decode_csv)).batch(3)
"""
We pass the build_model() function into a Keras Tuner optimization algorithm. Sev‐
eral algorithms are supported, but Bayesian optimization is an old standby that works
well for computer vision problems:
"""

import kerastuner as kt

tuner = kt.BayesianOptimization(
    build_model,
    objective = ['vak_accuracy', 'max'],
    max_trials = 10,
    num_initial_points = 2
    over_write = False ) #if true then it will overwrite the existing logs and checkpoints

"""
Here, we are specifying that our objective is to maximize the validation accuracy and
that we want the Bayesian optimizer to run 10 trials starting from 2 randomly chosen
seed points. The tuner can pick up where it left off, and we are asking Keras to do so
by telling it to reuse information learned in preexisting trials and not start with a
blank slate.
Having created the tuner, we can then run the search:
"""
tuner.search(
    train_dataset,
    validation_data = test_dataset,
    epochs = 5
    callbacks = [tf.keras.callbacks.EarlyStopping(patience = 2)]
    #patience is the number of epochs with no improvement after which training will be stopped
)
'''
At the end of the run, we can get the top N trials (the ones that ended with the highest
validation accuracy) using:
'''
topN = 2
for x in range(topN):
    print(tuner.get_best_parameters(topN)[x].values)
    print(tuner.get_best_models(topN)[x].summary)

# output will look like this {'lrate': 0.00017013245197465996, 'l2': 0.0, 'num_hidden': 64}

"""When we did hyperparameter tuning for the 5-flowers problem, we determined that
the best set of parameters was:
{'lrate': 0.00017013245197465996, 'l2': 0.0, 'num_hidden': 64}
The best validation accuracy obtained was 0.46."""

